<a href="https://colab.research.google.com/github/vtu28657/ait-record/blob/main/Task_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import inf

# Given distance matrix
d = np.array([
    [0, 10, 12, 11, 14],
    [10, 0, 13, 15, 8],
    [12, 13, 0, 9, 14],
    [11, 15, 9, 0, 16],
    [14, 8, 14, 16, 0]
])

# Parameters
iteration = 100
n_ants = 5
n_citys = 5
m = n_ants
n = n_citys
e = 0.5        # Evaporation rate
alpha = 1      # Pheromone importance
beta = 2       # Visibility importance

# Visibility = 1 / distance
visibility = 1 / d
visibility[visibility == inf] = 0

# Initialize pheromone trails
pheromone = 0.1 * np.ones((n, n))

# Route matrix for ants
rute = np.ones((m, n + 1))

for ite in range(iteration):
    rute[:, 0] = 1  # start from city 1 for all ants

    for i in range(m):
        temp_visibility = np.array(visibility)

        for j in range(n - 1):
            cur_loc = int(rute[i, j] - 1)

            # Set visibility of the current city to 0
            temp_visibility[:, cur_loc] = 0

            # Calculate probability components
            p_feature = np.power(pheromone[cur_loc, :], alpha)
            v_feature = np.power(temp_visibility[cur_loc, :], beta)
            combine_feature = p_feature * v_feature

            total = np.sum(combine_feature)
            probs = combine_feature / total if total != 0 else np.zeros(n)

            # Cumulative probability
            cum_prob = np.cumsum(probs)
            r = np.random.random_sample()
            city = np.nonzero(cum_prob > r)[0][0] + 1

            rute[i, j + 1] = city

        # Find the last unvisited city
        left = list(set(range(1, n + 1)) - set(rute[i, :-1]))
        if left:
            rute[i, -2] = left[0]
        rute[i, -1] = 1  # Return to start city

    # Calculate distance cost for all ants
    rute_opt = np.array(rute)
    dist_cost = np.zeros((m, 1))

    for i in range(m):
        s = 0
        for j in range(n):
            s += d[int(rute_opt[i, j]) - 1, int(rute_opt[i, j + 1]) - 1]
        dist_cost[i] = s

    # Find the best route
    dist_min_loc = np.argmin(dist_cost)
    dist_min_cost = dist_cost[dist_min_loc]
    best_route = rute[dist_min_loc, :]

    # Pheromone evaporation
    pheromone = (1 - e) * pheromone

    # Pheromone update based on distance
    for i in range(m):
        for j in range(n):
            dt = 1 / dist_cost[i]
            from_city = int(rute_opt[i, j]) - 1
            to_city = int(rute_opt[i, j + 1]) - 1
            pheromone[from_city, to_city] += dt

print('Routes of all the ants at the end:')
print(rute_opt)
print()
print('Best path:', best_route)
print('Cost of the best path:', int(dist_min_cost[0]))


Routes of all the ants at the end:
[[1. 2. 5. 3. 4. 1.]
 [1. 2. 5. 3. 4. 1.]
 [1. 2. 5. 3. 4. 1.]
 [1. 2. 5. 3. 4. 1.]
 [1. 2. 5. 3. 4. 1.]]

Best path: [1. 2. 5. 3. 4. 1.]
Cost of the best path: 52


/tmp/ipython-input-4068183285.py:24: RuntimeWarning: divide by zero encountered in divide
  visibility = 1 / d
/tmp/ipython-input-4068183285.py:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pheromone[from_city, to_city] += dt
